In [1]:
import os
import numpy as np
from scipy import stats
import pandas as pd
from pathlib import Path
from sklearn.model_selection import GridSearchCV, cross_validate,cross_val_score, cross_val_predict, LeaveOneGroupOut, KFold, ShuffleSplit, LeaveOneOut
from sklearn.base import TransformerMixin
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
from sklearn.pipeline import make_pipeline, Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import make_scorer
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

import shutil
import joblib

In [2]:

from dask.distributed import Client, LocalCluster
c = Client(processes = False,
    n_workers= 1,
    threads_per_worker=2
    )

from dask import delayed
import time

def inc(x):
    import time
    time.sleep(5)
    return x + 1

def dec(x):
    import time
    time.sleep(3)
    return x - 1

def add(x, y):
    import time
    time.sleep(7)
    return x + y

x = delayed(inc)(1)
y = delayed(dec)(2)
total = delayed(add)(x, y)
fut = c.compute(total)

In [5]:
# Just to demonstrate that dask is working
c.gather(fut)


3

In [6]:
c.cluster.scale(10)

In [7]:
import dask
dask.config.get

<function dask.config.get(key, default='__no_default__', config={'labextension': {'factory': {'module': 'dask_jobqueue', 'class': 'SLURMCluster', 'args': [], 'kwargs': {'local_directory': '/lscratch/$SLURM_JOB_ID', 'cores': 5, 'queue': 'quick', 'memory': '20g', 'processes': 1, 'threads': 10, 'job_extra': ['--constraint=10g', '--gres=lscratch:50']}}, 'default': {'workers': 1, 'adapts': {'minimum': 0, 'maximum': 10}}, 'initial': []}, 'temporary-directory': None, 'array': {'svg': {'size': 120}}, 'distributed': {'version': 2, 'scheduler': {'allowed-failures': 3, 'bandwidth': 100000000, 'blocked-handlers': [], 'default-data-size': 1000, 'events-cleanup-delay': '1h', 'idle-timeout': None, 'transition-log-length': 100000, 'work-stealing': True, 'worker-ttl': None, 'pickle': True, 'preload': [], 'preload-argv': [], 'default-task-durations': {}, 'dashboard': {'status': {'task-stream-length': 1000}, 'tasks': {'task-stream-length': 100000}, 'tls': {'ca-file': None, 'key': None, 'cert': None}}}, '

In [8]:
pv = 's_shaps_tot'
pred_var = pv
alpha_search=[0.01, 0.03, 0.1]
random_state=0
estimator='lasso'
pipe_names = ['clinical']

In [9]:
c.get_versions(check=True)

{'scheduler': {'host': [('python', '3.7.4.final.0'),
   ('python-bits', 64),
   ('OS', 'Linux'),
   ('OS-release', '3.10.0-862.14.4.el7.x86_64'),
   ('machine', 'x86_64'),
   ('processor', 'x86_64'),
   ('byteorder', 'little'),
   ('LC_ALL', 'en_US.UTF-8'),
   ('LANG', 'en_US.UTF-8'),
   ('LOCALE', 'en_US.UTF-8')],
  'packages': {'required': [('dask', '2.6.0'),
    ('distributed', '2.6.0'),
    ('msgpack', '0.6.1'),
    ('cloudpickle', '1.2.2'),
    ('tornado', '6.0.3'),
    ('toolz', '0.10.0')],
   'optional': [('numpy', '1.17.2'),
    ('pandas', '0.25.2'),
    ('bokeh', '1.3.4'),
    ('lz4', None),
    ('dask_ml', None),
    ('blosc', None)]}},
 'workers': {'inproc://10.2.2.122/17033/3': {'host': [('python',
     '3.7.4.final.0'),
    ('python-bits', 64),
    ('OS', 'Linux'),
    ('OS-release', '3.10.0-862.14.4.el7.x86_64'),
    ('machine', 'x86_64'),
    ('processor', 'x86_64'),
    ('byteorder', 'little'),
    ('LC_ALL', 'en_US.UTF-8'),
    ('LANG', 'en_US.UTF-8'),
    ('LOCALE', '

In [10]:
def col_picker(X, cols=None):
    if cols is not None:
        return X[:, cols]
    else:
        return X
    
def feature_stacker(X, n_stacks=1):
    dat = []
    for sn in range(n_stacks):
        start = sn * len(X)//n_stacks
        end = (sn+1) * len(X)//n_stacks
        dat.append(X[start:end])
    dat = np.vstack(dat).T
    return dat
    
def pass_through(y_true, y_pred):
    return y_pred


class RidgeTransformer(Ridge, TransformerMixin):
    def transform(self, X, *_):
        return self.predict(X)

    
class LassoTransformer(Lasso, TransformerMixin):
    def transform(self, X, *_):
        return self.predict(X)

    
class ElasticNetTransformer(ElasticNet, TransformerMixin):
    def transform(self, X, *_):
        return self.predict(X)
# num_cols = full_clin.columns[((full_clin.dtypes == 'float64') | (full_clin.dtypes == 'int64')) & (full_clin.isnull().sum() == 0)]
# X_df = full_clin.loc[(full_clin.session_number==1) & full_clin[pred_var].notnull(), num_cols]
# clinical_Xs = [ii for ii,ff in enumerate(X_df.columns.values) if ff in clinical_cols]


In [11]:
X_df = pd.read_csv('dask_minimal_data.csv')

In [12]:
clinical_cols = ['a', 'b', 'c']

In [13]:
y = X_df.loc[:, 'y'].values
X_df = X_df.loc[:, clinical_cols]
X = X_df.values


In [14]:

inner_cv = KFold(n_splits=5, shuffle=True, random_state=random_state)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=random_state)

if estimator == 'ridge':

    clinical_est = RidgeTransformer(fit_intercept=True)
elif estimator == 'lasso':

    clinical_est = LassoTransformer(fit_intercept=True)

elif estimator == 'elastic':

    clinical_est = ElasticNetTransformer(fit_intercept=True)
else:
    raise NotImplementedError


clinical_pipe = Pipeline(steps=[
                                ('scale', StandardScaler()),
                                ('rr', clinical_est)])

all_pipes = {
    'clinical':  {'pipeline':clinical_pipe, 'nX': X.shape[-1]},
}

pipes = {pn:all_pipes[pn] for pn in pipe_names}

n_stacks = len(pipes)
pred_union = FeatureUnion(transformer_list=[(k,v['pipeline']) for k,v in pipes.items()])

def feature_stacker(X):
    dat = []
    for sn in range(n_stacks):
        start = sn * len(X)//n_stacks
        end = (sn+1) * len(X)//n_stacks
        dat.append(X[start:end])
    dat = np.vstack(dat).T
    return dat
stack_transform = FunctionTransformer(feature_stacker, validate=False)

stacked_model = Pipeline(steps=[
    ('pred_union', pred_union),
    ('stacker', stack_transform),
    ('lin_regr', LinearRegression())
])

# Make parameter grid search 
parameters = {}
parameters[f'pred_union__clinical__rr__alpha'] = alpha_search
npc = np.product([len(v) for k,v in parameters.items()])
print(f"Running grid search across {npc} combinations of paramters")



Running grid search across 3 combinations of paramters


In [15]:
# Works without dask
stacked_gs = GridSearchCV(estimator=stacked_model, param_grid=parameters, cv=inner_cv, n_jobs=1, iid=True, pre_dispatch='2*n_jobs', verbose=10)
foo = cv_res = cross_validate(stacked_gs, X=X, y=y, cv=outer_cv, n_jobs=1, return_estimator=True, verbose=10)


[CV]  ................................................................
Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] pred_union__clinical__rr__alpha=0.01 ............................
[CV]  pred_union__clinical__rr__alpha=0.01, score=0.685, total=   0.0s
[CV] pred_union__clinical__rr__alpha=0.01 ............................
[CV]  pred_union__clinical__rr__alpha=0.01, score=0.524, total=   0.0s
[CV] pred_union__clinical__rr__alpha=0.01 ............................
[CV]  pred_union__clinical__rr__alpha=0.01, score=0.417, total=   0.0s
[CV] pred_union__clinical__rr__alpha=0.01 ............................
[CV]  pred_union__clinical__rr__alpha=0.01, score=-0.425, total=   0.0s
[CV] pred_union__clinical__rr__alpha=0.01 ............................
[CV]  pred_union__clinical__rr__alpha=0.01, score=0.366, total=   0.0s
[CV] pred_union__clinical__rr__alpha=0.03 ............................
[CV]  pred_union__clinical__rr__alpha=0.03, score=0.686, total=   0.0s
[CV] pred_union_

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elap

[CV]  pred_union__clinical__rr__alpha=0.03, score=0.476, total=   0.0s
[CV] pred_union__clinical__rr__alpha=0.03 ............................
[CV]  pred_union__clinical__rr__alpha=0.03, score=-0.039, total=   0.0s
[CV] pred_union__clinical__rr__alpha=0.03 ............................
[CV]  pred_union__clinical__rr__alpha=0.03, score=0.393, total=   0.0s
[CV] pred_union__clinical__rr__alpha=0.03 ............................
[CV]  pred_union__clinical__rr__alpha=0.03, score=0.453, total=   0.0s
[CV] pred_union__clinical__rr__alpha=0.03 ............................
[CV]  pred_union__clinical__rr__alpha=0.03, score=0.420, total=   0.0s
[CV] pred_union__clinical__rr__alpha=0.1 .............................
[CV] . pred_union__clinical__rr__alpha=0.1, score=0.477, total=   0.0s
[CV] pred_union__clinical__rr__alpha=0.1 .............................
[CV]  pred_union__clinical__rr__alpha=0.1, score=-0.043, total=   0.0s
[CV] pred_union__clinical__rr__alpha=0.1 .............................
[CV] 

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elaps

[CV]  pred_union__clinical__rr__alpha=0.03, score=0.391, total=   0.0s
[CV] pred_union__clinical__rr__alpha=0.1 .............................
[CV] . pred_union__clinical__rr__alpha=0.1, score=0.219, total=   0.0s
[CV] pred_union__clinical__rr__alpha=0.1 .............................
[CV] . pred_union__clinical__rr__alpha=0.1, score=0.373, total=   0.0s
[CV] pred_union__clinical__rr__alpha=0.1 .............................
[CV] . pred_union__clinical__rr__alpha=0.1, score=0.381, total=   0.0s
[CV] pred_union__clinical__rr__alpha=0.1 .............................
[CV] . pred_union__clinical__rr__alpha=0.1, score=0.657, total=   0.0s
[CV] pred_union__clinical__rr__alpha=0.1 .............................
[CV] . pred_union__clinical__rr__alpha=0.1, score=0.391, total=   0.0s
[CV] .................................... , score=0.471, total=   0.1s
[CV]  ................................................................
Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] pred_union__

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  15 out of  15 | elaps

In [16]:
# The first time it usually works
with joblib.parallel_backend('dask'):
    stacked_gs = GridSearchCV(estimator=stacked_model, param_grid=parameters, cv=inner_cv, n_jobs=1, iid=True, pre_dispatch='2*n_jobs', verbose=10)
    foo = cv_res = cross_validate(stacked_gs, X=X, y=y, cv=outer_cv, n_jobs=1, return_estimator=True, verbose=10)


[CV]  ................................................................
Fitting 5 folds for each of 3 candidates, totalling 15 fits[CV]  ................................................................

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend DaskDistributedBackend with 20 concurrent workers.
[Parallel(n_jobs=1)]: Using backend DaskDistributedBackend with 20 concurrent workers.
[Parallel(n_jobs=1)]: Using backend DaskDistributedBackend with 20 concurrent workers.
[Parallel(n_jobs=1)]: Using backend DaskDistributedBackend with 20 concurrent workers.
[Parallel(n_jobs=1)]: Using backend DaskDistributedBackend with 20 concurrent workers.


[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fitsFitting 5 folds for each of 3 candidates, totalling 15 fits



[Parallel(n_jobs=1)]: Using backend DaskDistributedBackend with 20 concurrent workers.


[CV] pred_union__clinical__rr__alpha=0.01 ............................
[CV] pred_union__clinical__rr__alpha=0.01 ............................
[CV] pred_union__clinical__rr__alpha=0.01 ............................
[CV] pred_union__clinical__rr__alpha=0.1 .............................
[CV] pred_union__clinical__rr__alpha=0.01 ............................
[CV] pred_union__clinical__rr__alpha=0.01 ............................
[CV] pred_union__clinical__rr__alpha=0.1 .............................
[CV] pred_union__clinical__rr__alpha=0.03 ............................
[CV] pred_union__clinical__rr__alpha=0.1 .............................
[CV] pred_union__clinical__rr__alpha=0.1 .............................
[CV] pred_union__clinical__rr__alpha=0.1 .............................
[CV] pred_union__clinical__rr__alpha=0.1 .............................
[CV] pred_union__clinical__rr__alpha=0.01 ............................
[CV] pred_union__clinical__rr__alpha=0.01 ............................
[CV] p

...

[Parallel(n_jobs=1)]: Done   6 out of  15 | elapsed:    4.6s remaining:    6.9s
[Parallel(n_jobs=1)]: Done   8 out of  15 | elapsed:    4.7s remaining:    4.1s
[Parallel(n_jobs=1)]: Done   2 out of  15 | elapsed:    4.7s remaining:   30.4s
[Parallel(n_jobs=1)]: Done  10 out of  15 | elapsed:    4.7s remaining:    2.3s
[Parallel(n_jobs=1)]: Done   4 out of  15 | elapsed:    4.7s remaining:   12.9s
[Parallel(n_jobs=1)]: Done   4 out of  15 | elapsed:    4.6s remaining:   12.5s
[Parallel(n_jobs=1)]: Done   6 out of  15 | elapsed:    4.7s remaining:    7.1s
[Parallel(n_jobs=1)]: Done   6 out of  15 | elapsed:    4.6s remaining:    6.8s
[Parallel(n_jobs=1)]: Done   8 out of  15 | elapsed:    4.7s remaining:    4.1s
[Parallel(n_jobs=1)]: Done   2 out of  15 | elapsed:    4.6s remaining:   29.8s



[CV]  pred_union__clinical__rr__alpha=0.01, score=0.374, total=   4.2s
[CV] . pred_union__clinical__rr__alpha=0.1, score=0.325, total=   4.0s
[CV]  pred_union__clinical__rr__alpha=0.01, score=0.430, total=   3.8s
[CV]  pred_union__clinical__rr__alpha=0.03, score=0.513, total=   4.0s[CV]  pred_union__clinical__rr__alpha=0.03, score=0.117, total=   4.0s

[CV]  pred_union__clinical__rr__alpha=0.01, score=0.262, total=   4.0s
[CV]  pred_union__clinical__rr__alpha=0.01, score=0.111, total=   3.9s
[CV] . pred_union__clinical__rr__alpha=0.1, score=0.549, total=   4.0s
[CV]  pred_union__clinical__rr__alpha=0.01, score=0.524, total=   4.0s
[CV]  pred_union__clinical__rr__alpha=0.01, score=0.415, total=   3.6s
[CV]  pred_union__clinical__rr__alpha=0.03, score=0.219, total=   3.8s
[CV]  pred_union__clinical__rr__alpha=0.01, score=0.367, total=   4.1s
[CV]  pred_union__clinical__rr__alpha=0.01, score=0.366, total=   4.0s
[CV] . pred_union__clinical__rr__alpha=0.1, score=0.219, total=   3.9s
[CV] 

[Parallel(n_jobs=1)]: Done  10 out of  15 | elapsed:    4.9s remaining:    2.4s
[Parallel(n_jobs=1)]: Done   4 out of  15 | elapsed:    4.7s remaining:   13.0s
[Parallel(n_jobs=1)]: Done  12 out of  15 | elapsed:    4.9s remaining:    1.2s
[Parallel(n_jobs=1)]: Done  12 out of  15 | elapsed:    4.9s remaining:    1.2s
[Parallel(n_jobs=1)]: Done   8 out of  15 | elapsed:    4.8s remaining:    4.2s
[Parallel(n_jobs=1)]: Done   6 out of  15 | elapsed:    4.8s remaining:    7.2s
[Parallel(n_jobs=1)]: Done   8 out of  15 | elapsed:    4.9s remaining:    4.3s
[Parallel(n_jobs=1)]: Done  10 out of  15 | elapsed:    4.9s remaining:    2.4s
[Parallel(n_jobs=1)]: Done   6 out of  15 | elapsed:    4.9s remaining:    7.3s
[Parallel(n_jobs=1)]: Done  12 out of  15 | elapsed:    4.9s remaining:    1.2s
[Parallel(n_jobs=1)]: Done   8 out of  15 | elapsed:    4.9s remaining:    4.3s
[Parallel(n_jobs=1)]: Done  10 out of  15 | elapsed:    4.9s remaining:    2.4s
[Parallel(n_jobs=1)]: Done  10 out of  1

[CV] .................................... , score=0.405, total=   5.2s
[CV] .................................... , score=0.231, total=   5.1s


CancelledError: _fit_transform_one-batch-c5f8b405fe6e4af8930aed47a595ab3d

In [46]:
# The scond and subsequent times, it dies
with joblib.parallel_backend('dask'):
    stacked_gs = GridSearchCV(estimator=stacked_model, param_grid=parameters, cv=inner_cv, n_jobs=1, iid=True, pre_dispatch='2*n_jobs', verbose=10)
    foo = cv_res = cross_validate(stacked_gs, X=X, y=y, cv=outer_cv, n_jobs=1, return_estimator=True, verbose=10)


[Parallel(n_jobs=1)]: Using backend DaskDistributedBackend with 20 concurrent workers.
[Parallel(n_jobs=1)]: Using backend DaskDistributedBackend with 20 concurrent workers.
[Parallel(n_jobs=1)]: Using backend DaskDistributedBackend with 20 concurrent workers.
[Parallel(n_jobs=1)]: Using backend DaskDistributedBackend with 20 concurrent workers.


[CV]  ................................................................
[CV]  ................................................................
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV]  ................................................................
Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV]  ................................................................
[CV] pred_union__clinical__rr__alpha=0.1 .............................
[CV]  ................................................................
Fitting 5 folds for each of 3 candidates, totalling 15 fitsFitting 5 folds for each of 3 candidates, totalling 15 fits

[CV] pred_union__clinical__rr__alpha=0.03 ............................
[CV] pred_union__clinical__rr__alpha=0.03 ............................
[CV] pred_union__clinical__rr__alpha=0.01 ............................
[CV] pred_union__clinical__rr__alpha=0.1 ....................

[Parallel(n_jobs=1)]: Using backend DaskDistributedBackend with 20 concurrent workers.
[Parallel(n_jobs=1)]: Using backend DaskDistributedBackend with 20 concurrent workers.


[CV] pred_union__clinical__rr__alpha=0.03 ............................
[CV] pred_union__clinical__rr__alpha=0.01 ............................
[CV] pred_union__clinical__rr__alpha=0.1 .............................
[CV] pred_union__clinical__rr__alpha=0.01 ............................
[CV] pred_union__clinical__rr__alpha=0.01 ............................
[CV] pred_union__clinical__rr__alpha=0.1 .............................
[CV] pred_union__clinical__rr__alpha=0.1 .............................
[CV] pred_union__clinical__rr__alpha=0.1 .............................
[CV] pred_union__clinical__rr__alpha=0.1 .............................
[CV] pred_union__clinical__rr__alpha=0.03 ............................
[CV] pred_union__clinical__rr__alpha=0.03 ............................
[CV] pred_union__clinical__rr__alpha=0.01 ............................
[CV] pred_union__clinical__rr__alpha=0.1 .............................
[CV] pred_union__clinical__rr__alpha=0.01 ............................
[CV] p

/data/nielsond/python/envs/dask/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:530: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)
distributed.worker - WARNING -  Compute Failed
Function:  <joblib._dask.Batch object at 0x2aab687cdbd0>
args:      (array([[ 0. ,  0. ,  0. ],
       [ 4. ,  0. , 18. ],
       [ 0. ,  0. ,  1. ],
       [12. ,  1. , 40. ],
       [ 5. ,  6. , 16. ],
       [ 9. ,  5. , 29.7],
       [19. ,  6. , 27. ],
       [ 7. ,  1. , 38. ],
       [ 1. ,  1. , 12. ],
       [ 4. ,  3. , 12. ],
       [15. , 10. , 61. ],
       [ 0. ,  0. ,  2. ],
       [13. ,  4. , 28. ],
       [ 0. ,  0. , 15. ],
       [20. ,  1. , 68. ],
       [ 4. ,  0. ,  6. ],
       [ 3. ,  1. , 13. ],
       [24. ,  1. , 60. ],
       [ 6. ,  7. , 31. ],
       [ 7. ,  3. , 

...

[Parallel(n_jobs=1)]: Done   8 out of  15 | elapsed:    3.9s remaining:    3.5s
[Parallel(n_jobs=1)]: Done   8 out of  15 | elapsed:    3.8s remaining:    3.3s
[Parallel(n_jobs=1)]: Done  10 out of  15 | elapsed:    4.0s remaining:    2.0s
[Parallel(n_jobs=1)]: Done  10 out of  15 | elapsed:    3.8s remaining:    1.9s
[Parallel(n_jobs=1)]: Done  12 out of  15 | elapsed:    4.0s remaining:    1.0s
[Parallel(n_jobs=1)]: Done  10 out of  15 | elapsed:    3.9s remaining:    1.9s
[Parallel(n_jobs=1)]: Done  12 out of  15 | elapsed:    3.9s remaining:    1.0s
[Parallel(n_jobs=1)]: Done  12 out of  15 | elapsed:    3.9s remaining:    1.0s
/data/nielsond/python/envs/dask/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:530: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)
distribut

[CV]  pred_union__clinical__rr__alpha=0.03, score=0.393, total=   2.7s[CV]  pred_union__clinical__rr__alpha=0.01, score=0.326, total=   2.5s

[CV]  pred_union__clinical__rr__alpha=0.01, score=0.415, total=   2.2s
[CV]  pred_union__clinical__rr__alpha=0.01, score=0.685, total=   2.2s[CV]  pred_union__clinical__rr__alpha=0.03, score=0.415, total=   2.5s[CV]  pred_union__clinical__rr__alpha=0.03, score=0.476, total=   2.2s


[CV] . pred_union__clinical__rr__alpha=0.1, score=0.453, total=   2.1s
[CV] . pred_union__clinical__rr__alpha=0.1, score=0.415, total=   1.9s
[CV]  pred_union__clinical__rr__alpha=0.01, score=-0.425, total=   2.7s
[CV]  pred_union__clinical__rr__alpha=0.03, score=-0.425, total=   2.7s
[CV]  pred_union__clinical__rr__alpha=0.01, score=0.453, total=   2.7s
[CV]  pred_union__clinical__rr__alpha=0.03, score=0.340, total=   1.9s


[Parallel(n_jobs=1)]: Done   3 out of   5 | elapsed:    4.4s remaining:    2.9s


[CV] .................................... , score=0.471, total=   4.4s
[CV] .................................... , score=0.637, total=   4.2s


CancelledError: _fit_transform_one-batch-d73abb665c6b4f5d8ce93f453918a4cb